# Voice AI Agent with Object Detection

In [ ]:
## install Import necessary library
import cv2
import os 
import sys
import re
import time
import json
import base64
import requests
import numpy as np
import matplotlib.pyplot as plt
from openai import OpenAI
from yolov8.yolov8 import YOLOv8
from yolov8.utils import class_names
import pyttsx3



## Init camera and YOLOv8 

In [ ]:
# Init camera
video_cap = cv2.VideoCapture(0)
video_cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
video_cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Create an inference session using the Vitis AI execution provider
#input_model_file = r"onnx_utils\DetectionModel_int.onnx"
#config_file_path = r"onnx_utils\vaip_config.json"
#model = YOLOv8(input_model_file, config_file_path, 0.25) # probability threshold: 0.25. Notes: high probability threshold will casue the model failing in detecting object.


In [ ]:
# capture one image

success, frame = video_cap.read()
frame = frame[:, 280:-280, ::-1]

print(frame.shape)
plt.imshow(frame)
plt.imsave('temp\out_for_llava.jpg', frame)

#ret = model.detect_objects(frame[:, :, ::-1])
#detected_img = model.draw_detections(frame)
#plt.imshow(detected_img)
#plt.imsave('temp\\result.jpg', detected_img)

In [ ]:
# # parse label
# def parse_ret(ret, origin_img):
#     parsed_data = []
#     parsed_img = {}
#     for i in range(len(ret[0])):
#         x1, y1, x2, y2 = ret[0][i].astype(np.int32)
#         object_dict = {
#             "Index": "ThisIsSearchIndex_"+str(i),
#             "coordination": [str((x1+x2)//2), str((y1+y2)//2)],
#             "class": class_names[ret[2][i]],
#         }
#         parsed_img["ThisIsSearchIndex_"+str(i)] = origin_img[y1:y2, x1:x2, :]
#         parsed_data.append(object_dict)
#     return parsed_data, parsed_img


# parsed_result, parsed_img = parse_ret(ret, frame[:, :, ::1])
# print(parsed_result)

## ENTER YOUR QUERY HERE

In [ ]:
query = "please analyze the picture for me and tell me a story?"

### This our enhanced prompt here. 
This prompt is aimed at link the objects in image with the detection results from yolov8 so that the model can ground target object precisely.

In [ ]:

prompt_1 = f"""
You are an excellent AI agent who can read pictures and tell stories

The group you are targeting are children and novices who don't understand anything

please give me a output as a text format finally

Remember:no less than 300 words
"""

prompt_2 = f"""

You need to imagine a story based on the information in the image and tell it

Include as much information as possible from all images

###### USER QUERY ###### 

User query: {query}. You must return one answer.

Please analyze this query and picture step by step, but there is no need to provide an analysis process
###### OUTPUT ######
Combining the above requirements,please use a paragraph to tell me a complete and interesting story.
Answer: 
"""
prompt = prompt_1 + prompt_2

## Call LLM to reason
The LLM api is called in format of openai. And the model we use here is the "llava-7b".

In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
# 你需要传入进的图片路径
image_path = "temp\\out_for_llava.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="")
messages = []
first_turn = {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": f"{prompt}",
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
                }
            ]
        }
messages.append(first_turn)
payload = {
    "model": "llava-v1.5-7b-llamafile",
    "messages": f"{json.dumps(messages, indent=4)}",
}
response = requests.post("http://127.0.0.1:1234/v1/chat/completions", json=payload)
print(response)
response.json()

In [ ]:
content = str(response.json()['choices'][0]['message']['content'])
print(content)

# trans to voice playback


In [ ]:
# 文字转语音的初始化设置
engine = pyttsx3.init()
engine.setProperty('rate',150)
engine.setProperty('volume',0.4)

In [ ]:


engine.say(content)
engine.runAndWait()